## Value Investing Stock Screener 



In this project, stocks in the S&P 500 index are screened and given an overall "value" score which takes into account their **Price-to-Earnings**, **Price-to-Book-Value**, and **Enterprise-Value-to-Free-Cash-Flow** ratios. 


The **Finnhub** API is used to retrieve current stock-market data. 

In [78]:
import finnhub
import math
import pandas as pd
import time

### Retrieving Data 
First, the stocks in the S&P 500 are retrieved using the Index's Constituents endpoint (https://finnhub.io/docs/api/indices-constituents).

Then, the ratios of interest are retrieved through the Basic Financials endpoint (https://finnhub.io/docs/api/company-basic-financials).

In [79]:
from api_secrets import FINNHUB_API_KEY

# Setup client
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# Retrieve all the stock tickers in the S&P 500 Index
sp500 = finnhub_client.indices_const(symbol = "^GSPC")['constituents']


In [80]:
# setup dictionary which has the stock symbols as keys, and their ratios as the values (also stored as a dictionary)
ratios = {}

# loop over stocks in index, adding key-value pairs to dict
for symbol in sp500[:30]:
    res = finnhub_client.company_basic_financials(symbol, 'metric')
    try:
        pe = res['metric']['peNormalizedAnnual']
        pb = res['metric']['pbAnnual']
        ev_to_cashflow = res['metric']['currentEv/freeCashFlowAnnual']
        ratios[symbol] = {'pe':pe, 'pb':pb, 'ev/cashflow': ev_to_cashflow}
    except KeyError:
        print('missing values detected for: ', symbol)
    
    time.sleep(1) # necessary to deal with the 60-calls-per-minute rate limit that comes with the free api. 
                  # change according to your rate limit
    
# the code above will take at least 500 seconds to run with the free rate limit

### Wrangling the Data and Creating a DataFrame
Below the dictionary is filtered to remove stocks with missing ratio values. Missing values for the ratios indicate that a stock has negative earnings or cash flows. A value-oriented investor will likely look down upon such stocks, so it is reasonable that they would be removed from a list of value-oriented stock recommendations. 

In [31]:
# needs to use a copy of ratios so that loop indices work properly
filtered_ratios = dict(ratios)

for symbol in ratios.keys():
    if(ratios[symbol]['pb'] == None or ratios[symbol]['pe'] == None or ratios[symbol]['ev/cashflow'] == None):
        del filtered_ratios[symbol]
        
# replace original ratios with filtered ratios
ratios = filtered_ratios


A pandas DataFrame is made to represent the data. Each observation (row) in the DataFrame contains the name of the stock, as well its financial ratios that were obtained earlier. In addition, each ratio value has a corresponding percentile value indicating its percentile relative to the entire column (this will be computed later). Finally, each stock will be given an overall score from 0-100. 

In [32]:
# setup columns of dataframe
value_metrics_dataframe_cols = [
    'symbol',
    'price', 
    'number of shares to buy',
    'pe ratio', 
    'pe percentile',
    'pb ratio', 
    'pb percentile',
    'ev-to-cf ratio',
    'ev-to-cf percentile',
    'overall score (higher is better)'
]

# setup dataframe
value_metrics_dataframe = pd.DataFrame(columns = value_metrics_dataframe_cols)

# add rows incrementally to the end of the dataframe
for symbol in ratios.keys():
    price = finnhub_client.quote(symbol)['c']
    value_metrics_dataframe.loc[len(value_metrics_dataframe.index)] = [
        symbol, 
        price,
        'N/A',
        ratios[symbol]['pe'],
        'N/A',
        ratios[symbol]['pb'],
        'N/A',
        ratios[symbol]['ev/cashflow'],
        'N/A',
        'N/A'
    ]
    
n_head = 5 # used to indicate the number of elements to be printed by the dataframe.head() function
value_metrics_dataframe.head(n_head)


,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,COO,330.67,N/A,39.59756,N/A,2.28637,N/A,39.11909,N/A,N/A
1,IQV,204.89,N/A,39.53713,N/A,6.43031,N/A,30.64521,N/A,N/A
2,CME,168.16,N/A,27.28481,N/A,2.26854,N/A,91.62803,N/A,N/A
3,VTRS,11.13,N/A,202.24100,N/A,0.64983,N/A,15.29912,N/A,N/A
4,WBA,37.36,N/A,5.42249,N/A,1.32150,N/A,18.25741,N/A,N/A


### Percentile and Overall Score Calculation
The percentile of each ratio is computed below, by using `scipy.stats.percentileofscore()`.

In [33]:
from scipy.stats import percentileofscore as score
prefixes = ['pe', 'pb', 'ev-to-cf']

# calculate percentiles for all the ratios, put them in the percentile column
for ratio in prefixes:
    entire_column = value_metrics_dataframe[f'{ratio} ratio'] # entire column for current ratio
    for row in value_metrics_dataframe.index:
        val = value_metrics_dataframe.loc[row, f'{ratio} ratio']
        # set the current percentile column as the percentile of the corresponding ratio value. 
        # division by 100 is to simplify excel formatting 
        value_metrics_dataframe.loc[row, f'{ratio} percentile'] = score(entire_column, val)/100 
        
        
        
value_metrics_dataframe.head(n_head)



,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,COO,330.67,N/A,39.59756,0.9,2.28637,0.45,39.11909,0.8,N/A
1,IQV,204.89,N/A,39.53713,0.85,6.43031,0.75,30.64521,0.55,N/A
2,CME,168.16,N/A,27.28481,0.55,2.26854,0.4,91.62803,0.95,N/A
3,VTRS,11.13,N/A,202.24100,1.0,0.64983,0.05,15.29912,0.2,N/A
4,WBA,37.36,N/A,5.42249,0.15,1.32150,0.2,18.25741,0.3,N/A


The overall score is defined as $1 - \mu$, where $\mu$ is equal to the mean of the percentiles.

In [34]:
from statistics import mean

for row in value_metrics_dataframe.index:
    percentiles = []
    for ratio in prefixes:
        percentiles.append(value_metrics_dataframe.loc[row, f'{ratio} percentile'])
    
    # mean(percentiles) gives average percentile. higher percentile values indicate a worse value, therefore
    # this mean will be inverted (subtracted from 1) for the overall score calculation
    value_metrics_dataframe.loc[row, 'overall score (higher is better)'] = 1 - mean(percentiles) 

value_metrics_dataframe.head(n_head)


,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,COO,330.67,N/A,39.59756,0.9,2.28637,0.45,39.11909,0.8,0.283333
1,IQV,204.89,N/A,39.53713,0.85,6.43031,0.75,30.64521,0.55,0.283333
2,CME,168.16,N/A,27.28481,0.55,2.26854,0.4,91.62803,0.95,0.366667
3,VTRS,11.13,N/A,202.24100,1.0,0.64983,0.05,15.29912,0.2,0.583333
4,WBA,37.36,N/A,5.42249,0.15,1.32150,0.2,18.25741,0.3,0.783333


### Selecting the 20 best stocks
Below the data is sorted so that the stocks with a higher overall score are at the top. This would mean that they are undervalued relative to other investments. Then, only the top 20 stocks are selected, by slicing the dataframe. 

In [42]:
value_metrics_dataframe.sort_values('overall score (higher is better)', ascending = False, inplace = True)
value_metrics_dataframe.reset_index(inplace = True, drop = True)

value_metrics_dataframe = value_metrics_dataframe[:20]

value_metrics_dataframe.head(n_head)

,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,BIO,420.49,118,2.90135,0.05,0.90383,0.15,2.67873,0.05,0.916667
1,WDC,31.55,1584,5.23383,0.1,0.78847,0.1,11.02672,0.1,0.9
2,WBA,37.36,1338,5.42249,0.15,1.32150,0.2,18.25741,0.3,0.783333
3,UHS,140.89,354,11.75096,0.2,1.78193,0.3,23.75252,0.35,0.716667
4,BMY,71.95,694,19.12777,0.4,4.36578,0.6,14.81006,0.15,0.616667


### Calculating the Number of Shares to Buy
The program now asks for the user's input to calculate the size of the positions and the number of shares to buy. 

In [71]:
portfolio_size = input('Enter the size of your portfolio: ')

# loop while float conversion is not successful due to type difference (ValueError)
while(True):
    try:
        portfolio_size = float(portfolio_size)
        break # exit loop, everything successful
    except ValueError:
        portfolio_size = input('Input was not numerical, please try again \nEnter the size of your portfolio: ')
        continue # continue looping until there is a numerical input


Enter the size of your portfolio: 
Input was not numerical, please try again 
Enter the size of your portfolio: 1000000


The size of each position is calculated below. Each stock is given an equal weighting when calculating the number of shares to buy.

In [72]:
position_size = portfolio_size/len(value_metrics_dataframe.index)

for row in value_metrics_dataframe.index:
    # math.floor is used to remove fractional shares
    # the number of stocks to buy = (position size) / (price of the stock)
    value_metrics_dataframe.loc[row, 'number of shares to buy'] =  math.floor(position_size/value_metrics_dataframe.loc[row, 'price'])

In [73]:
value_metrics_dataframe # print the dataframe in full

,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,BIO,420.49,118,2.90135,0.05,0.90383,0.15,2.67873,0.05,0.916667
1,WDC,31.55,1584,5.23383,0.1,0.78847,0.1,11.02672,0.1,0.9
2,WBA,37.36,1338,5.42249,0.15,1.32150,0.2,18.25741,0.3,0.783333
3,UHS,140.89,354,11.75096,0.2,1.78193,0.3,23.75252,0.35,0.716667
4,BMY,71.95,694,19.12777,0.4,4.36578,0.6,14.81006,0.15,0.616667
5,GOOGL,88.23,566,15.90595,0.3,4.69577,0.65,16.19659,0.25,0.6
6,VTRS,11.13,4492,202.24100,1.0,0.64983,0.05,15.29912,0.2,0.583333
7,NTRS,88.49,565,12.16191,0.25,1.64296,0.25,209.18140,1.0,0.5
8,IR,52.25,956,36.31085,0.75,2.36838,0.5,25.51460,0.4,0.45
9,MCO,278.62,179,23.66319,0.45,18.98060,0.9,30.37979,0.5,0.383333


### Exporting the Data to Excel
The xlsxwriter module is used to export our dataframe to excel.

In [74]:
import xlsxwriter

# setup excel writer and link sheet with dataframe
writer = pd.ExcelWriter('value_screener.xlsx', engine = 'xlsxwriter')
value_metrics_dataframe.to_excel(writer, sheet_name = 'Top 20 Undervalued Stocks', index = False)


Below we will setup the templates necessary for formatting our columns properly.

In [75]:
background_color = '#C4A484'
font_color = '#000000'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

The formats defined above are applied to the appropriate columns, and the names of the columns are set. 

In [76]:
column_formats  = {
    'A': ['Symbol', string_template],
    'B': ['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['PE ratio', float_template],
    'E':['PE percentile', percent_template],
    'F':['PB ratio', float_template],
    'G':['PB percentile', percent_template],
    'H':['EV/CF ratio', float_template],
    'I':['EV/CF percentile', percent_template],
    'J':['Overall Score (higher is better)', percent_template]
         }

for column in column_formats.keys():
    writer.sheets['Top 20 Undervalued Stocks'].set_column(f'{column}:{column}', 22, column_formats[column][1]) # set format for column
    writer.sheets['Top 20 Undervalued Stocks'].write(f'{column}1', column_formats[column][0], string_template) # set name for column

writer.sheets['Top 20 Undervalued Stocks'].set_column('J:J', 25, column_formats['J'][1]); # overwrite format to fit name


In [77]:
# save the excel output
writer.save()

C:\Users\billy\AppData\Local\Temp\ipykernel_13456\2836777261.py:2: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


### The list of undervalued stocks has now been exported to *value_screener.xlsx* !

**Note**: the workflow for this project was inspired by the *Algorithmic Trading in Python* course (https://github.com/nickmccullum/algorithmic-trading-python). However, there are notable differences in the approach used for obtaining stock-market data (a different API was used for this purpose), and several improvements are made to increase the robustness of the code. For example, a loop was added to continuously check that the user input is numeric when calculating position size. Missing data is also dealt with differently (rows with missing data are removed in this project). 